## Molecular evolution of the influenza virus

The text file **influenza_HA_dna_sequences.fasta** contains the complete DNA sequence of the hemagglutinin (HA) gene of 841 influenza viruses samples collected between 1968 and 2005. Hemagglutinin is a protein that mediates the virus entry into cells and is a target of antibodies produced by the human host that neutralize the virus. To avoid recognition by the immune system, the virus changes its hemagglitinin gene sequence over time. Analyze these sequence data to answer the following questions:

* How do mutations accumulate as a function of time? That is, how does the number of single nucleotide differences between the first sample (A/Aichi/2/1968) and samples from a given year vary as a function of time? Make the graph.

* What fraction of the HA gene has changed as a function of time?

* What is the genetic variation (average pairwise distance) between the strains from the same year?

* How long would we have to wait for the population to accumulate the same number of genetic differences that typically separate co-circulating strains?